In [ ]:
import pandas as pd 
import tiledbsoma as soma
import tiledb

import pickle

In [ ]:
soma_join_id_block

In [ ]:
# read soma
ctx = soma.SOMATileDBContext(tiledb_ctx=tiledb.Ctx(
       
    ))




print("reading soma...")

soma_experiment = soma.Experiment.open("s3://pai-scrnaseq/sctx_gui/corpora/human_v6.0.5", context=ctx)

obs_df = soma_experiment.obs.read().concat().to_pandas()
obs_df.shape

In [ ]:
soma_joinid_to_barcode = obs_df[obs_df["barcode"].isin(l)][["soma_joinid", "barcode"]]

soma_joinid_to_barcode

soma_joinid_to_barcode.to_csv("scref_train_v6_soma_joinid_barcode.csv")

In [ ]:
soma_joinid_to_barcode = pd.read_csv("scref_train_v6_soma_joinid_barcode.csv")

In [ ]:
soma_joinid_to_barcode['soma_joinid'].nunique()

In [ ]:
ref_train_studies = obs_df_ref[obs_df_ref["barcode"].isin(l)]["study_name"].values.tolist()
v6_train_studies = obs_df[obs_df["barcode"].isin(l)]["study_name"].values.tolist()

print(set(v6_train_studies).difference(set(ref_train_studies)))

In [ ]:
sam_barcodes = sam_embeddings['barcode'] 

v6_barcodes = obs_df['barcode']

sam_embeddings[sam_embeddings['barcode'].isin(set(sam_barcodes).difference(set(v6_barcodes)))]['disease_name'].value_counts()


In [ ]:
sam_barcodes_not_in_v6_train = set(sam_barcodes).difference(set(v6_train_barcodes))
sam_barcodes_not_in_v6_train

In [ ]:
sam_embeddings[sam_embeddings['barcode'].isin(sam_barcodes_not_in_v6_train)]['study_name'].value_counts()

In [ ]:
soma_ids = obs_df[obs_df["barcode"].isin(l)]["soma_joinid"].values.tolist()
len(soma_ids)

In [ ]:
obs_df[['soma_joinid', 'barcode']].to_csv("scmark_soma_joinid_to_barcode.csv")

In [ ]:
# filter out nuclei
print(obs_df["cells_or_nuclei"].value_counts())

obs_df_sub = obs_df[obs_df["cells_or_nuclei"] != "nuclei"]

# filter out low quality cells
print((obs_df_sub["nnz"] > 300).sum())

obs_df_sub = obs_df_sub[obs_df_sub["nnz"] > 300]


In [ ]:
print(obs_df_sub['study_name'].value_counts()[:100].index)

In [ ]:
# read json
import json

with open('/home/ubuntu/bascvi/data/v6_train_downsample.json') as f:
    downsample_dict = json.load(f)

In [ ]:
# downsample by sample

def down_func(group):
    if group.name not in downsample_dict.keys():
        downsample_dict[group.name] = 0

    if group.shape[0] >= downsample_dict[group.name]:
        group_sub = group.sample(n=downsample_dict[group.name])
    else:
        group_sub = group

    return group_sub

obs_df_sub_samp = obs_df_sub.groupby('dataset_name').apply(down_func).reset_index(drop = True)
print(obs_df_sub_samp.shape[0])
print(obs_df_sub_samp['dataset_name'].value_counts())

In [ ]:
pd.set_option('display.max_rows', 500)


print(obs_df_sub_samp['study_name'].value_counts(dropna=False).head(150))

In [ ]:
# holdout Wu and Qian
obs_df_sub_samp_holdout = obs_df_sub_samp[~obs_df_sub_samp['study_name'].isin(['external_qian_cellres_2020_32561858', 'external_wu_natgenet_2021_34493872', 'external_azizi_cell_2018_29961579'])]


In [ ]:
obs_df_sub_samp_holdout.study_name.value_counts()

In [ ]:
import pickle
with open('../data/v6_train_barcodes.pkl', 'wb') as f:
    pickle.dump(obs_df_sub_samp_holdout['barcode'].values.tolist(), f)

In [ ]:
len(obs_df_sub_samp_holdout['barcode'].values.tolist())